In [2]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from datetime import date, datetime, timedelta
import sqlalchemy
import pymysql
import openpyxl
import glob
from ast import literal_eval
from collections import Counter
from tqdm.auto import tqdm
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import pyarrow as pa
from dask.distributed import Client

In [3]:
load_dotenv()

hyperlink_path = 'json_completed_hyperlinks_update.csv'
parent_dir = os.getenv('dir')
data_dir = os.path.join(parent_dir,'data_update')

df = pd.read_csv(hyperlink_path, header=None)
df.head()
df.columns = ['ParseID','Hyperlink']
hyperlinks = df['Hyperlink'].tolist()

def foldername(hyperlink):
    hyperlink = hyperlink.split('/')[-1]
    return hyperlink[0:-8]
def providers_path(folder):
    return os.path.join(data_dir,folder,folder+'_providers.csv')

folder_names= [foldername(hyperlink) for hyperlink in hyperlinks]
provider_files = [providers_path(folder_name) for folder_name in folder_names]

In [4]:
client = Client(n_workers=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 4,Total memory: 3.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50935,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 3.86 GiB
Comm: tcp://127.0.0.1:50947,Total threads: 4
Dashboard: http://127.0.0.1:50948/status,Memory: 3.86 GiB
Nanny: tcp://127.0.0.1:50938,


In [6]:
ddf = dd.read_parquet('D://Vignesh/Capstone/data_update/*/*_merge.parquet', 
                     columns=['billing_type','billing_code','negotiated_rates','ccn'], engine='pyarrow')

ddf


,billing_type,billing_code,negotiated_rates,ccn
npartitions=428,,,,
,object,object,float64,object
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [7]:
%%time

ddf = ddf.drop_duplicates(ignore_index=True)
# ddf.to_csv("D://Vignesh/Capstone/combined/export.csv")

CPU times: total: 15.6 ms
Wall time: 17.2 ms


In [11]:
n = ddf['billing_type'].nunique().compute()
n

In [8]:
unquie_types = ddf['billing_type'].unique().compute()
bt_max = ddf['billing_type'].str.len().max().compute()
bc_max = ddf['billing_code'].str.len().max().compute()
ccn_max = ddf['ccn'].str.len().max().compute()

In [9]:
print('The max length for ccn is :{0}'.format(ccn_max))

The max length for ccn is :6


In [ ]:
(host, user, password, port, database) = (os.getenv('host'), os.getenv('user'), os.getenv('passwd'), os.getenv('port'), os.getenv('database'))

def get_connection():
    return sqlalchemy.create_engine(
        url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, port, database
        )
    )

engine = get_connection()

Task exception was never retrieved
future: <Task finished name='Task-887' coro=<Client._gather.<locals>.wait() done, defined at c:\ProgramData\Anaconda3\lib\site-packages\distributed\client.py:2002> exception=AllExit()>
Traceback (most recent call last):
  File "c:\ProgramData\Anaconda3\lib\site-packages\distributed\client.py", line 2011, in wait
    raise AllExit()
distributed.client.AllExit


In [10]:
table= 'rates'
columns = ['billing_type', 'billing_code', 'negotiated_rates', 'ccn']
sqltypes_rates = {'billing_type': sqlalchemy.types.VARCHAR(length=8), 'billing_code': sqlalchemy.types.VARCHAR(length=7), 
                  'negotiated_rates': sqlalchemy.types.FLOAT(length=32), 'ccn': sqlalchemy.types.VARCHAR(length=6)}

TypeError: __init__() got an unexpected keyword argument 'length'

In [ ]:
ddf.head()

In [ ]:
ddf.to_sql(name=table,con=engine, if_exists='append', index=False, chunksize=100, dtype=sqltypes_rates)